# 1. Cài đặt thuật toán tiến trước, thuật toán Viterbi, và thuật toán Baum-Welch.

In [192]:
import numpy as np
import pandas as pd
import random as rd

## 1.1. Thuật toán tiến trước

[https://github.com/adeveloperdiary/HiddenMarkovModel/blob/master/part2/forward.py](https://github.com/adeveloperdiary/HiddenMarkovModel/blob/master/part2/forward.py)

In [193]:
''' The forward algorithm: 
Given an HMM with:
transition probability table A and emission probability table (observation ikelihood) B 
return the probability (likelihood) of a observation sequence O'''

# all the variables in this function are np.arrays
def forward_algorithm(observation, transition_prob, emission_prob, initial_distribution, vocabulary):
    'observations has length of T, the number of different states N'
    # create a probability matrix alpha
    T = observation.shape[0]
    N = transition_prob.shape[0]
    alpha = np.zeros((N, T))
    
    # initialization
    id = np.where(vocabulary == observation[0]) # get the index of the first observation in the vocabulary

    alpha[: , 0] = initial_distribution * emission_prob[:, id[0][0]]

    # recursion
    for i in range(1, T):
        id = np.where(vocabulary == observation[i])
        for j in range(N):
            alpha[j, i] = alpha[:, i - 1].dot(transition_prob[:, j]) * emission_prob[j, id[0][0]]

    # termination
    forward_prob = np.sum(alpha[:, T - 1])
    return (forward_prob, alpha)


# obs = np.array((3, 1, 3))
# trans = np.array([[.5, .5], [.4, .6]])
# ems = np.array([[.5, .4, .1], [.2, .4, .4]])
# ini = np.array((.2, .8))
# vcb = np.array((1, 2, 3))
# print(forward_algorithm(obs, trans, ems, ini, vcb))

## 1.2. Thuật toán Viterbi

[https://github.com/adeveloperdiary/HiddenMarkovModel/blob/master/part4/Viterbi.py](https://github.com/adeveloperdiary/HiddenMarkovModel/blob/master/part4/Viterbi.py)

In [194]:
'''Viterbi algorithm:
Given an HMM with:
transition probability table A and emission probability table (observation ikelihood) B 
a sequence of observations O
find the most sequence of states Q
'''
def viterbi_algorithm(observation, transition_prob, emission_prob, initial_distribution, vocabulary):
    T = observation.shape[0]
    N = transition_prob.shape[0]
    omega = np.zeros((N, T))
    
    # initialization
    id = np.where(vocabulary == observation[0])
    
    omega[:, 0] = initial_distribution * emission_prob[:, id[0][0]] # initialize the same as the forward algorithm

    prev = np.zeros((N, T))
    prev[:, 0] = 0

    temp = np.zeros(N)

    # recursion
    for i in range(1, T):
        # find the index of the observation in the vocabulary
        id = np.where(vocabulary == observation[i])
        for j in range(N):    
            # the same as forward probability
            for k in range(N):
                temp[k] = omega[k, i - 1] * transition_prob[k, j] * emission_prob[j, id[0][0]]
                
            # the most probable state given previous state at time i    (1)
            prev[j, i] = np.argmax(temp)

            # the probability of the most probable state                (2)
            omega[j, i] = np.max(temp)

    # print(omega)

    # termination
    best_path_prob = np.max(omega[:, T - 1])

    # path array: the most probable sequence of states for the observations sequence.
    path = np.zeros(T, dtype= int)

    # the most probable state at the last time step
    path[T - 1] = int(np.argmax(omega[:, T - 1]))

    # backtracking
    for i in range(T - 2, -1, -1):
        path[i] = int(prev[path[i + 1], i + 1])

    return (path)

# print(viterbi_algorithm(obs, trans, ems, ini, vcb))

## 1.3. Thuật toán Baum - Welch

In [195]:
# backward probability
def backward_algorithm(observation, transition_prob, emission_prob, initial_distribution, vocabulary):
    # initialization
    T = observation.shape[0]
    N = transition_prob.shape[0]
    beta = np.zeros((N, T))

    # initialization
    for i in range(N):
        beta[i, T - 1] = 1
    
    # recursion
    for t in range(T - 2, -1, -1):
        for i in range(N):
            # find the index of the (t + 1)-th observation
            id = np.where(vocabulary == observation[t + 1])
            beta[i, t] = 0

            for j in range(N):
                beta[i, t] = beta[i, t] + transition_prob[i, j] * emission_prob[j, id[0][0]] * beta[j, t + 1]
    
    # termination
    id = np.where(vocabulary == observation[0])
    print(initial_distribution.shape, emission_prob.shape, beta[:, 0].shape)
    backward_prob = np.dot(np.dot(initial_distribution , emission_prob[:, id[0][0]]), beta[:, 0])
    
    return (backward_prob, beta)

In [196]:
# baum-welch algorithm: from the observation sequence O and the set of possible states in the HMM, learn the HMM parameters A (alpha) and B (beta)
def baum_welch_algorithm(observation, transition_prob, emission_prob, initial_distribution, vocabulary, n_iter=100):
    T = observation.shape[0]
    N = transition_prob.shape[0]
    
    for n in range(n_iter):
        (f_pr, alpha) = forward_algorithm(observation, transition_prob, emission_prob, initial_distribution, vocabulary)
        (b_pr, beta) = backward_algorithm(observation, transition_prob, emission_prob, initial_distribution, vocabulary)
        xi = np.zeros((N, N, T - 1))

        for t in range(T - 1):
            id = np.where(vocabulary == observation[t + 1])

            denominator = np.dot(np.dot(alpha[ : , t], transition_prob) * emission_prob[:, id[0][0]], beta[:, t + 1])
            for i in range(N):
                numerator = alpha[i, t] * transition_prob[i, :] * emission_prob[:, id[0][0]] * beta[:, t + 1]
                xi[i, :, t] = numerator / denominator

        gamma = np.sum(xi, axis=1)
        alpha = np.sum(xi, 2) / np.sum(gamma, axis=1).reshape((-1, 1))

        # Add additional T'th element in gamma
        gamma = np.hstack((gamma, np.sum(xi[:, :, T - 2], axis=0).reshape((-1, 1))))

        K = emission_prob.shape[1]
        denominator = np.sum(gamma, axis=1)
        for l in range(K):
            emission_prob[:, l] = np.sum(gamma[:, observation == l], axis=1)

        emission_prob = np.divide(emission_prob, denominator.reshape((-1, 1)))

    return (transition_prob, emission_prob)

# 2. Bài toán: 
Khi làm quản trò, anh Huy thường sử dụng 2 viên xúc xác khác nhau. Viên đầu tiên là một viên xúc xắc cân bằng, mọi mặt đều có cùng xác suất. Viên thứ hai là một viên xúc xắc lỗi, khi tung sẽ có 50% xác suất ra mặt số 6 và 10% xác suất ra mỗi mặt còn lại. Mỗi lần tung, anh sẽ chọn 1 trong 2 viên xúc xắc này để tung. Người chơi không thể biết anh đã tung viên nào, chỉ biết được lần tung đó ra mặt nào. Ngoài ra, nếu ở lần tung này, anh Huy sử dụng viên xúc xắc cân bằng, thì có 80% khả năng anh sẽ tiếp tục sử dụng viên xúc xắc này cho lần tung tiếp theo (20% còn lại anh sẽ đổi sang dùng viên lỗi). Con số này là 30% đối với viên lỗi (70% đổi sang dùng viên cân bằng)

## a) Mô hình hóa tình huống trên bằng một mô hình Markov ẩn. Cho biết các tham số của mô hình này.
Mô hình Markov ẩn được xây dựng:
- Tập trạng thái `Q = {q0: cân bằng, q1: lỗi}`.
- Ma trận chuyển trạng thái `A = [[0.8, 0.2], [0.7, 0.3]]` (theo thứ tự `a00, a01, a10, a11`).
- Tập quan sát O gồm các trạng thái được lấy từ tập `V = {1, 2, 3, 4, 5, 6}`.
- Ma trận B (các giá trị observation likelihoods): 
`B = [[P(1|q0) = 1/6, P(2|q0) = 1/6, P(3|q0) = 1/6, P(4|q0) = 1/6, P(5|q0) = 1/6, P(6|q0) = 1/6],
 [P(1|q1) = 0.1, P(2|q1) = 0.1, P(3|q1) = 0.1, P(4|q1) = 0.1, P(5|q1) = 0.1, P(6|q1) = 0.5]]`
- Phân phối ban đầu `Pi = [0.5, 0.5]`

## b) Sinh ngẫu nhiên một chuỗi T = 100 lần tung theo đúng mô tả trên.

In [197]:
'A is the transition probability matrix'
A = [[0.8, 0.2], [0.7, 0.3]]

'B is the observation likelihoods matrix'
B = [[1/6, 1/6, 1/6, 1/6, 1/6, 1/6],[.1, .1, .1, .1, .1, .5]]

'D is the number on the faces of the dice'
D = [1,2,3,4,5,6]

'initial distribution'
Pi = np.array((0.5, 0.5))

def generate(T: int):
    dice = rd.choice([0,1])     # choose a random dice with equal probability 0.5
    res = list()                # list of observations
    dices = list()              # hidden states
    for i in range(0, T):
        dices.append(dice)
        temp1 = rd.choices(D, B[dice])
        res.append(temp1[0])
        temp2 = rd.choices([0,1], A[dice])
        dice = temp2[0]
    # print('Hidden states: ', dices)
    return res

'Generate a sequence of T = 100 observations'
obs = np.array(generate(100))   # observation seq.
trans = np.array(A)  # transition prob.
emiss = np.array(B)  # emission prob.
vocab = np.array(D)  # vocabulary
print('Observation sequence: ', obs)

Observation sequence:  [4 5 6 1 6 6 6 4 5 6 6 5 6 3 1 1 6 2 3 6 3 4 1 4 2 4 4 6 6 6 2 6 2 3 1 3 1
 1 6 3 3 5 3 6 4 2 6 1 4 4 1 5 2 2 3 6 3 5 4 3 3 6 5 5 6 5 1 6 1 6 6 1 3 4
 5 4 5 6 4 2 3 4 4 6 6 4 5 1 2 6 6 6 6 2 2 5 3 1 2 2]


## c) Sử dụng thuật toán Viterbi để dự đoán viên xúc xắc được dùng cho mỗi lần tung. Độ chính xác của dự đoán này là bao nhiêu? Hãy lặp lại thí nghiệm này nhiều lần nếu cần thiết. Báo cáo và nhận xét kết quả thu được.

In [198]:
state_guess = viterbi_algorithm(obs, trans, emiss, Pi, vocab)
print(state_guess)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


## d) Giả sử bạn là một người chơi, hãy sử dụng thuật toán Baum-Welch để ước lượng các tham số cho mô hình Markov ẩn. Hãy lặp lại thí nghiệm nhiều lần nếu cần thiết. Báo cáo và nhận xét kết quả thu được.

In [199]:
# transition probabilities
alpha = np.identity(2)

# emission probabilities
beta = np.ones((2, 6))/6

alpha, beta = baum_welch_algorithm(obs, alpha, beta, Pi, vocab)
print('alpha = ', alpha)
print('beta = ', beta)

(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)


C:\Users\DELL\AppData\Local\Temp/ipykernel_14400/478258657.py:17: RuntimeWarning: invalid value encountered in true_divide
  xi[i, :, t] = numerator / denominator


(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2,)
(2,) (2, 6) (2